In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *

In [2]:
from kilosort import run_kilosort, io
import spikeinterface.extractors as se

### Load Configs

In [34]:
from config import dir_config, s_probe_path

raw_dir = Path(dir_config.data.raw)
sorting_dir = Path(dir_config.data.sorting)
compiled_dir = Path(dir_config.data.compiled)

session_metadata = pd.read_csv(Path(compiled_dir, "sessions_metadata.csv"), index_col=None)

### Utils functions

In [67]:
def convert_nsx_to_bin(nsx_path, nsx_name, bin_path, bin_name):
	nsx_data, N, c, s, fs, probe_path = None, None, None, None, None, None

	nsx_file = Path(nsx_path, f"{nsx_name}.ns5")
	if nsx_file.exists():
		nsx_data = se.read_blackrock(file_path=Path(nsx_path, nsx_name), stream_id="5")
		filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(nsx_data, bin_path, data_name=f"{bin_name}.bin", dtype=np.int16, chunksize=60000, export_probe=True, probe_name="probe.prb")
		print(f"{N}, {c}, {s}, {fs}, {probe_path}")
	else:
		print(f"File {nsx_file} does not exist")
	return nsx_data, N, c, s, fs, probe_path

## Covert blackrock (NSX) data to bin

In [74]:
for idx, session_id in enumerate(session_metadata["session_id"]):
	# Construct the path to the raw and binary file
	nsx_path = Path(raw_dir) / session_id
	nsx_name = session_id
	bin_path = Path(sorting_dir) / session_id
	bin_name = session_id
	bin_file = bin_path / f"{bin_name}.bin"

	try:
		# check if raw data exists
		if not nsx_path.exists():
			print(f"Raw data does not exist for: \t {session_id}")
			continue

		# Check if the binary file already exists
		if not bin_file.exists():
			print(f"Converting {session_id} to binary")
			# Call the conversion function
			nsx_data, N, c, s, fs, probe_path = convert_nsx_to_bin(nsx_path=str(nsx_path), nsx_name=nsx_name, bin_path=str(bin_path), bin_name=bin_name)
		else:
			print(f"Bin file exists for: \t {bin_name}")
	except Exception as e:
		print(f"Error: {e}")
		continue

Bin file exists for: 	 210126_GP_JP
Raw data does not exist for: 	 210205_GP_JP
Raw data does not exist for: 	 210210_GP_JP
Raw data does not exist for: 	 210211_GP_JP
Raw data does not exist for: 	 210216_GP_JP
Raw data does not exist for: 	 210217_GP_JP
Raw data does not exist for: 	 210224_GP_JP
Raw data does not exist for: 	 210302_GP_JP
Raw data does not exist for: 	 210305_GP_JP
Raw data does not exist for: 	 210311_GP_JP
Raw data does not exist for: 	 210312_GP_JP
Raw data does not exist for: 	 210315_GP_JP
Raw data does not exist for: 	 210316_GP_JP
Raw data does not exist for: 	 210317_GP_JP
Raw data does not exist for: 	 210318_GP_JP
Raw data does not exist for: 	 210323_GP_JP
Raw data does not exist for: 	 210413_GP_JP
Raw data does not exist for: 	 210524_GP_JP
Raw data does not exist for: 	 210525_GP_JP
Raw data does not exist for: 	 210528_GP_JP
Raw data does not exist for: 	 210601_GP_JP
Raw data does not exist for: 	 210602_GP_JP
Raw data does not exist for: 	 210603_GP

## Run Kilosort

#### KS settings

In [36]:
drift_correction = True

settings_s_probe_ucla = {
	"fs": 30000,
	"batch_size": 120000,
	"min_template_size": 30,
	"nearest_templates": 6,
	"n_chan_bin": 18,
}
settings_s_probe_uw = {
	"fs": 30000,
	"batch_size": 120000,
	"min_template_size": 30,
	"nearest_templates": 6,
	"n_chan_bin": 19,
}
if drift_correction:
	settings_s_probe_ucla["nblocks"] = 1
	settings_s_probe_uw["nblocks"] = 1


s_probe = io.load_probe(s_probe_path)
assert s_probe is not None, "No probe information exported by SpikeInterface"

In [45]:
for idx, session_id in enumerate(session_metadata["session_id"]):
	try:
		session_path = Path(sorting_dir, session_id)
		bin_file = f"{str(session_path)}/{session_id}.bin"
		if session_path.exists():
			# importing correct settings and probe
			if session_metadata["probe"][idx] == "s-probe":
				subject_id = session_metadata["subject"][idx]
				if subject_id == "Jasper":
					settings = settings_s_probe_ucla
					probe = s_probe
				elif subject_id == "Taz":
					settings = settings_s_probe_uw
					probe = s_probe

				# run kilosort
				ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = run_kilosort(
					settings=settings,
					probe=probe,
					filename=bin_file,
				)
			elif session_metadata["probe"][idx] == "single":
				print(f"Single electrode probe won't be processed with kilosort: {session_id}")
		else:
			print(f"Session not found: {session_id}")
			continue

	except Exception as e:
		print(f"Error in {session_id}: {e}")
		continue

/mnt/prior-data/sorting/210126_GP_JP/210126_GP_JP.bin
